In [25]:
import pandas as pd
import numpy as np

df = pd.read_excel("../../data/raw/Cocci Case Counts by County and Month, 2001-2022.xlsx")
#relabel the columns to be correct
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
#remove unnessacary columns
columns = df.columns.tolist()
new_columns = columns[:-2]
df = df[new_columns]
#relabel the counties to match the formatting of all our other datasets
df['County'] = df['County'].str.title()
#relabel the other columns to be more accurate
rename_dict = {
    'Year and\nMonth of\nEstimated\nOnset': "YearMonth",
    'Case\nCount': 'Cases'
}
df = df.rename(columns=rename_dict)
#split the yearmonth columnn into two sepearate columns
df[['Year', 'Month']] = df['YearMonth'].str.split('_', expand=True)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
#create the datecolumn
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2) + '-01')
#remove the yearmont column as it is unnessacary
df = df.drop('YearMonth', axis=1)
#figure out what sc is
df.tail()

/home/intellect/miniconda3/envs/harbinger/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,County,Cases,Year,Month,Date
15307,Yuba,SC*,2022,8,2022-08-01
15308,Yuba,SC*,2022,9,2022-09-01
15309,Yuba,SC*,2022,10,2022-10-01
15310,Yuba,SC*,2022,11,2022-11-01
15311,Yuba,SC*,2022,12,2022-12-01


In [15]:
df.columns

Index(['County', 'Year and\nMonth of\nEstimated\nOnset', 'Case\nCount'], dtype='object', name=0)

In [26]:
df['Cases'].unique()

array([0, 2, 1, 3, 5, 10, 4, 7, 6, 8, 15, 13, 16, 9, 11, 'SC*', 12, 17,
       21, 19, 18, 14, 26, 20, 31, 29, 44, 95, 78, 52, 54, 35, 79, 36, 64,
       50, 90, 59, 48, 67, 65, 33, 27, 25, 22, 24, 30, 56, 61, 53, 39, 49,
       43, 40, 32, 34, 96, 113, 128, 91, 72, 62, 63, 51, 45, 87, 88, 73,
       86, 46, 23, 28, 38, 55, 37, 42, 81, 134, 58, 93, 114, 132, 98, 109,
       100, 41, 47, 76, 60, 141, 151, 224, 148, 85, 175, 140, 117, 123,
       150, 142, 176, 195, 147, 126, 106, 124, 138, 146, 139, 127, 149,
       102, 74, 66, 194, 214, 257, 184, 103, 68, 120, 77, 75, 89, 104,
       143, 101, 111, 188, 129, 118, 125, 83, 80, 69, 208, 284, 442, 286,
       215, 217, 105, 121, 135, 167, 164, 243, 304, 332, 326, 253, 198,
       237, 178, 116, 163, 152, 144, 185, 137, 158, 179, 70, 57, 115, 264,
       323, 413, 307, 193, 94, 174, 191, 335, 365, 440, 474, 338, 331,
       207, 168, 171, 245, 223, 269, 226, 306, 282, 254, 314, 276, 359,
       228, 271, 213, 281, 262, 366, 339, 251, 301,